In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import scipy.stats
%matplotlib inline

## ANÁLISE CHI-SQUARE: CONFIANÇA NAS REDES SOCIAIS POR FAIXA ETÁRIA

In [ ]:
df = pandas.read_csv('dataset.csv')
print(f"Dataset original: {len(df)} registros")

## Limpando possíveis valores incorretos

In [ ]:
df_clean = df[df['Age'] >= 18].copy()
print(f"Após limpeza (idade >= 18): {len(df_clean)} registros")

In [ ]:
print(df_clean['Trust in Social Media'].value_counts())

## Criação dos grupos de idade

In [ ]:
def categorizar_idade(idade):
    if 18 <= idade <= 30:
        return 1  # Jovens
    elif 31 <= idade <= 50:
        return 2  # Adultos  
    elif idade >= 51:
        return 3  # Idosos
        
df_clean['FAIXAETARIA'] = df_clean['Age'].apply(categorizar_idade)

## Análise binária

In [ ]:
def categorizar_confianca(confianca):
    if confianca in ['Trust a lot', 'Trust somewhat']:
        return 1  # Com confiança
    elif confianca == 'Do not trust':
        return 0  # Sem confiança

df_clean['CONFIANCA'] = df_clean['Trust in Social Media'].apply(categorizar_confianca)

## Limpando NULLs

In [ ]:
sub2 = df_clean.dropna(subset=['FAIXAETARIA', 'CONFIANCA']).copy()
print(f"Dataset final: {len(sub2)} registros")

## Crosstab

In [ ]:
ct1 = pandas.crosstab(sub2['CONFIANCA'], sub2['FAIXAETARIA'])
print("\nTabela de Contingência:")
print(ct1)

## Soma todos os valores por coluna e pego o porcentual por coluna

In [ ]:
colsum = ct1.sum(axis=0)
colpct = ct1/colsum

print("\nPercentuais por coluna (faixa etária):")
print(colpct)

print('\nValor de Qui-quadrado, Valor p, valores esperados')
cs1 = scipy.stats.chi2_contingency(ct1)
print(cs1)

sub2["FAIXAETARIA"] = sub2["FAIXAETARIA"].astype('category')

sub2['CONFIANCA'] = pandas.to_numeric(sub2['CONFIANCA'], errors='coerce')

## Gráficos

In [ ]:
seaborn.catplot(x="FAIXAETARIA", y="CONFIANCA", data=sub2, kind="bar", ci=None)
plt.xlabel('Faixa Etária (1=Jovens, 2=Adultos, 3=Idosos)')
plt.ylabel('Proporção de Confiança nas Redes Sociais')
plt.title('Confiança nas Redes Sociais por Faixa Etária')
plt.show()

## Descritivos

In [ ]:
print("\n" + "="*50)
print("INTERPRETAÇÃO DOS RESULTADOS")
print("="*50)

chi2_stat, p_value, dof, expected = cs1
print(f"Teste geral (todas as faixas etárias):")
print(f"Chi-square: {chi2_stat:.4f}")
print(f"P-valor: {p_value:.6f}")
print(f"Significativo (p < 0.05): {'SIM' if p_value < 0.05 else 'NÃO'}")
print(f"Graus de liberdade: {dof}")

if p_value < 0.05:
    print("\nHá diferenças estatisticamente significativas na confiança entre as faixas etárias!")
else:
    print("\nNão há diferenças estatisticamente significativas na confiança entre as faixas etárias.")

In [ ]:
print(f"\n" + "="*50)
print("ESTATÍSTICAS DESCRITIVAS")
print("="*50)

faixas = {1: 'Jovens (18-30)', 2: 'Adultos (31-50)', 3: 'Idosos (51+)'}
for codigo, nome in faixas.items():
    subset = sub2[sub2['FAIXAETARIA'] == codigo]
    com_confianca = len(subset[subset['CONFIANCA'] == 1])
    total = len(subset)
    pct = (com_confianca / total * 100) if total > 0 else 0
    print(f"{nome}: {com_confianca}/{total} ({pct:.1f}%) têm confiança nas redes sociais")